In [2]:
print('linhscscs')

linhscscs


In [5]:
pip install pandas numpy underthesea


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import re,string
from underthesea import word_tokenize
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
#from gensim.models import KeyedVectors
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
from timeit import default_timer as timer
import math
import warnings
warnings.filterwarnings('ignore')

OSError: [WinError 127] The specified procedure could not be found

In [14]:
# data_dir = "/content/drive/MyDrive/xlnntn/data/"
# en_sents = open(data_dir + 'en_sents', "r",encoding="utf-8").read().splitlines()
# vi_sents = open(data_dir + 'vi_sents', "r",encoding="utf-8").read().splitlines()
# raw_data = {
#         "en": [line for line in en_sents[:254000]], # Only take first 170000 lines
#         "vi": [line for line in vi_sents[:254000]], # 254000
#     }
# df = pd.DataFrame(raw_data, columns=["en", "vi"])
# print(len(en_sents))
# df.head()

This dataset provides a set of 254,090 tuples containing an English source sentence, its Vietnamese human translation and we take 170000 set for training and evaluation

In [ ]:
import pandas as pd
import os

# Đường dẫn tới thư mục chứa dữ liệu
data_dir = "data12/"

# Danh sách lưu trữ các câu
en_sents = []
vi_sents = []

# Đọc dữ liệu từ các thư mục con
for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)
    if os.path.isdir(subdir_path):
        en_file = os.path.join(subdir_path, 'data.en')
        vi_file = os.path.join(subdir_path, 'data.vi')

        if os.path.isfile(en_file):
            with open(en_file, "r", encoding="utf-8") as file:
                en_sents.extend(file.read().splitlines())

        if os.path.isfile(vi_file):
            with open(vi_file, "r", encoding="utf-8") as file:
                vi_sents.extend(file.read().splitlines())

# Đọc dữ liệu từ các tập tin train, dev, tst
for file_name in ['train.en', 'train.vi', 'dev.en', 'dev.vi', 'tst.en', 'tst.vi']:
    file_path = os.path.join(data_dir, file_name)

    if file_name.endswith('.en'):
        if os.path.isfile(file_path):
            with open(file_path, "r", encoding="utf-8") as file:
                en_sents.extend(file.read().splitlines())

    elif file_name.endswith('.vi'):
        if os.path.isfile(file_path):
            with open(file_path, "r", encoding="utf-8") as file:
                vi_sents.extend(file.read().splitlines())

# Kiểm tra độ dài của các danh sách
print(f"Number of English sentences: {len(en_sents)}")
print(f"Number of Vietnamese sentences: {len(vi_sents)}")

# Đảm bảo hai danh sách có cùng độ dài (nếu cần thiết)
min_length = min(len(en_sents), len(vi_sents))
en_sents = en_sents[:min_length]
vi_sents = vi_sents[:min_length]

# Tạo DataFrame
raw_data = {
    "en": en_sents,
    "vi": vi_sents,
}
df = pd.DataFrame(raw_data, columns=["en", "vi"])

# Hiển thị thông tin
df.head()


Number of English sentences: 4125698
Number of Vietnamese sentences: 4125865


,en,vi
0,I have to go to sleep.,Tôi phải đi ngủ.
1,Today is June 18th and it is Muiriel's birthday!,"Hôm nay là ngày 18 tháng sáu, và cũng là ngày ..."
2,Muiriel is 20 now.,Bây giờ Muiriel được 20 tuổi.
3,"The password is ""Muiriel"".","Mật mã là ""Muiriel""."
4,I will be back soon.,Tôi trở lại mau.


## Data Preprocessing

In [16]:
df.isna().sum()

en    0
vi    0
dtype: int64

In [17]:
# def preprocessing(df):
#   df["en"] = df["en"].apply(lambda ele: ele.translate(str.maketrans('', '', string.punctuation))) # Remove punctuation
#   df["vi"] = df["vi"].apply(lambda ele: ele.translate(str.maketrans('', '', string.punctuation)))
#   df["en"] = df["en"].apply(lambda ele: ele.lower()) # convert text to lowercase
#   df["vi"] = df["vi"].apply(lambda ele: ele.lower())
#   df["en"] = df["en"].apply(lambda ele: ele.strip())
#   df["vi"] = df["vi"].apply(lambda ele: ele.strip())
#   df["en"] = df["en"].apply(lambda ele: re.sub("\s+", " ", ele))
#   df["vi"] = df["vi"].apply(lambda ele: re.sub("\s+", " ", ele))

#   return df

# df = preprocessing(df)
# df.head()

Tokenzing for english which is source language by tokenizer of basic english and vietnamese which is target language by Underthesea library

In [18]:
# Create source and target language tokenizer.
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'vi'

# Place-holders
token_transform = {}
vocab_transform = {}

# Tokenize for vietnames by underthesea
def vi_tokenizer(sentence):
    tokens = word_tokenize(sentence)
    return tokens

token_transform[SRC_LANGUAGE] = get_tokenizer('basic_english')
token_transform[TGT_LANGUAGE] = get_tokenizer(vi_tokenizer)

# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    for index,data_sample in data_iter:
        yield token_transform[language](data_sample[language])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = df.iterrows()
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set UNK_IDX as the default index. This index is returned when the token is not found.
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

## Model Defination

In [19]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #Check whether running on gpu or cpu

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float = 0.1,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()

        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [20]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Define the parameters of the model, instantiate the same and the loss function which is the cross-entropy loss and the optmizer used for training is Adam with β1 = 0.9, β2 = 0.98 and epsilon = 1e−9.

In [21]:
print(len( vocab_transform[SRC_LANGUAGE]))
print(len( vocab_transform[TGT_LANGUAGE]))

190216
343842


In [22]:
torch.manual_seed(0)
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8 # embed_dim must be divisible by num_heads
FFN_HID_DIM = 512
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 4
NUM_DECODER_LAYERS = 4
DROP_OUT = 0.1

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM,DROP_OUT)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

Define collate function that convert batch of raw strings into batch tensors that can be fed directly into model.

In [23]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor




# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []

    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Split data to train and test set

In [24]:
# Split data to tran test set
split_ratio = 0.9
split = round(df.shape[0]* split_ratio)
train = df.iloc[:split]
train_ds = list(zip(train['en'],train['vi']))
valid = df.iloc[split:]
val_ds = list(zip(valid['en'],valid['vi']))

Define training and evaluation loop that will be called for each epoch with Gradient accumulation which is a technique where you can train on bigger batch sizes than your machine would normally be able to fit into memory. This is done by accumulating gradients over several batches, and only stepping the optimizer after a certain number of batches have been performed.

In [25]:
from torch.utils.data import DataLoader

accumulation_steps = 5

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    val_los = 0
    train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    optimizer.zero_grad()
    for i, (src, tgt) in enumerate(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss = loss / accumulation_steps # Normalize our loss (if averaged)
        loss.backward()

        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step() # Now we can do an optimizer step
            optimizer.zero_grad() # Reset gradients tensor

        losses += loss.item()

    return losses / len(train_dataloader)

def evaluate(model):
    model.eval()
    losses = 0

    #val_iter = valid.iterrows()
    val_dataloader = DataLoader(val_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss = loss / accumulation_steps # Normalize our loss (if averaged)
        losses += loss.item()

    return losses / len(val_dataloader)

Define an early stopping function to avoid the model from overfit

In [26]:
class EarlyStopping():
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:
                self.early_stop = True

Training model

In [3]:
early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)
NUM_EPOCHS = 3
history = {
        "loss": [],
        "val_los": []
        }

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    history['loss'].append(train_loss)
    history['val_los'].append(val_loss)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    # Early Stopping
    early_stopping(train_loss, val_loss)
    if early_stopping.early_stop:
        print("We are at epoch:", epoch)
        break

NameError: name 'EarlyStopping' is not defined

Traning and Validate plotting

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history['loss'], label = "loss")
plt.plot(history['val_los'], label = "Val loss")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss vs. No. of epochs')

**Inference**

In [ ]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
# Saving model
torch.save(transformer.state_dict(), "viEn_transformer1.pth")

Testing model by randomly take 10 samples from the rest of the dataset

In [ ]:
from random import randint
rand = [randint(170000,200000) for i in range(10)]
test_set = [
        [en_sents[i] for i in rand],
        [vi_sents[i] for i in rand]]

for i in range(len(test_set[0])):
    print('Input English sentence:', test_set[0][i])
    print('Actual Vietnamese Translation:', test_set[1][i])
    print('Predicted Vietnamese Translation:', translate(transformer, test_set[0][i]))
    print("====> " , translate(transformer, 'end the war'))
    print("\n")

In [ ]:
print("====> " , translate(transformer, 'I am Linh, who are you?'))

First Segment:
"The sun was setting over the horizon, casting a golden glow across the sky. The gentle breeze rustled through the leaves of the trees. Birds were returning to their nests as the day turned into night."

Second Segment:
"In the quiet of the evening, the sounds of the city began to fade away. The streetlights flickered on, one by one, illuminating the sidewalks. People started to head home, ending their day."

Third Segment:
"As the stars began to appear, the night sky became a canvas of twinkling lights. The tranquility of the night settled in, bringing a sense of calm. It was the perfect end to a beautiful day."

Đoạn văn tiếng Việt:
Đoạn đầu tiên:
"Mặt trời đang lặn dần về phía chân trời, ánh sáng vàng óng ánh lên bầu trời. Cơn gió nhẹ thổi qua những tán lá của cây cối. Những chú chim trở về tổ khi ngày chuyển dần thành đêm."

Đoạn thứ hai:
"Trong sự yên tĩnh của buổi tối, âm thanh của thành phố bắt đầu lắng xuống. Những đèn đường lần lượt sáng lên, chiếu sáng vỉa hè. Mọi người bắt đầu trở về nhà, kết thúc một ngày làm việc."

Đoạn thứ ba:
"Khi những vì sao bắt đầu xuất hiện, bầu trời đêm trở thành một bức tranh ánh sáng lấp lánh. Sự yên bình của đêm buông xuống, mang đến cảm giác thư thái. Đó là cái kết hoàn hảo cho một ngày đẹp trời."

In [ ]:
import re

def split_text(text):
    # Tách văn bản thành các phần trước và sau ký tự đặc biệt, bao gồm ký tự đặc biệt
    parts = re.split(r'([,.!?;:])', text)
    return parts


def process_parts(parts):
    processed_parts = []

    # Vòng lặp qua từng phần của văn bản đã tách
    for part in parts:

        # Ở đây có thể thêm bất kỳ xử lý nào bạn cần
        if part in [',','.','!','?',';',':']:
          processed_parts.append(part)
        elif part ==  "":
          processed_parts.append(part)
        else :
          part = translate(transformer, part)
          processed_parts.append(part)



    # Ghép các phần lại với nhau
    result = ''.join(processed_parts)
    return result

# Ví dụ
text = "In the quiet of the evening, the sounds of the city began to fade away. The streetlights flickered on, one by one, illuminating the sidewalks. People started to head home, ending their day."
parts = split_text(text)  # Tách văn bản
result = process_parts(parts)  # Xử lý các phần đã tách
print(result)  # Output: "I am Linh, who are you? Let's go!"
out = "Trong sự yên tĩnh của buổi tối, âm thanh của thành phố bắt đầu lắng xuống. Những đèn đường lần lượt sáng lên, chiếu sáng vỉa hè. Mọi người bắt đầu trở về nhà, kết thúc một ngày làm việc."
print(out)
